In [1]:
from utils.datasets import get_dataset
train_dataset, val_dataset, test_dataset = get_dataset()

100%|██████████| 170498071/170498071 [00:05<00:00, 30372095.64it/s]


Extracting data/cifar10\cifar-10-python.tar.gz to data/cifar10
Files already downloaded and verified


In [5]:
print(path for path in train_dataset)

<generator object <genexpr> at 0x000001C72D593ED0>
